In [ ]:
import numpy as np
import yaml

with open("../data/D_germline_ex.yaml", 'r') as stream:
    try:
        y = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Build a dictionary of the YAML keyed by state name.
state_d = {}
for s in y['states']:
    state_d[s['name']] = s

bases = list('ACGT')
insert_names = ['insert_left_'+x for x in bases]
dummy_names = ['dummy_D_'+str(i) for i in range(5)]

In [ ]:
def stack_rows(array_list):
    return np.concatenate([np.array([a]) for a in array_list])

def extract_transitions(d, names):
    def get_with_default(n):
        if n in d:
            return d[n]
        else:
            return 0.
    v = np.array([get_with_default(n) for n in names])
    return v

#### NTI transition and emission matrices

![](https://cloud.githubusercontent.com/assets/112708/20118401/1d070a30-a5b9-11e6-8f0f-272950064215.png)

In [ ]:
in_landing = extract_transitions(state_d['init']['transitions'], insert_names)

T = stack_rows([
        extract_transitions(state_d[n]['transitions'], insert_names) 
        for n in insert_names])

# Note the transpose at the end here, which is because we are indexing
# observed states on the rows and hidden states on the columns.
E = stack_rows([
        np.array([state_d[n]['emissions']['probs'][b] for b in bases]) 
        for n in insert_names]).T

out_landing = stack_rows([
        extract_transitions(state_d[i_n]['transitions'], dummy_names) 
        for i_n in insert_names])

# These are the flexbounds for the NTI region. The left one is equal to the right 
# flexbounds of the previous gene. Because that describes the nucleotide positions 
# of uncertainty just after the end of the germline gene match, this describes the 
# first possible nucleotide positions of the NTI region, in this example 4 and 5.
left_flexbounds = (4, 5)
right_flexbounds = (6, 8)
# Relpos for the dummy D gene.
right_relpos = 5
emission_indices = [0, 1, 0, 2, 3, 0, 1, 1, 1, 3, 2, 3, 3]
used_emission_indices = emission_indices[left_flexbounds[0]:right_flexbounds[1]]

for o in [in_landing, T, E, out_landing, used_emission_indices]:
    print o

![](https://cloud.githubusercontent.com/assets/112708/19703331/49f810a4-9ab8-11e6-8eaa-9b2766bc547e.png)

In [ ]:
# Following the image in the issue literally:
all_A = []
for t in range(len(used_emission_indices)):
    y_t = used_emission_indices[t]
    A = np.zeros((t+1, 4))
    if t > 0:
        A[:t, :] = all_A[t-1].dot(T) * E[y_t, :]
    A[t, :] = in_landing * E[y_t, :]
    all_A.append(A)

However, we don't need all of the contents of the As.
We only need to start at positions 3 and 4, which because of the setup of used_emission_indices are the first two entries
(i.e. the left flex.)

In [ ]:
left_flex = left_flexbounds[1]-left_flexbounds[0]
sub_A = [A[:(left_flex+1), :] for A in all_A]
sub_A

We want the matrix that has been run out to one position before the LHS of the right flex (but we subtract off `left_flexbounds[0]` because we're starting the computation at that position; see definition of `used_emission_indices`).
We then right multiply by the column vector that corresponds to the entrypoint of the D gene. 

In [ ]:
for i in range(0,right_flexbounds[1]-right_flexbounds[0]+1):
    print sub_A[i+right_flexbounds[0]-left_flexbounds[0]-1].dot(out_landing[:,i+right_flexbounds[0]-right_relpos])